In [1]:
from __future__ import print_function
import numpy as np
np.random.seed(1337)  # for reproducibility

from keras.models import Sequential,Model
from keras.layers import Dense, Dropout, Activation, Flatten,Input
from keras.layers import Convolution2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.resnet50 import ResNet50
import matplotlib.pyplot as plt
%matplotlib inline

Using Theano backend.
Using gpu device 0: GRID K520 (CNMeM is disabled, cuDNN 5005)


In [2]:
batch_size = 8
nb_classes = 9
#image_size=(218,303)
#input_shape=(3,218,303)
#image_size=(128,128)
#input_shape=(3,128,128)
image_size=(224,224)
input_shape=(3,224,224)
classes=["chicken","ostrich",'bluebird','finch','frog','salemander','cobra','bird','flamingo']

In [3]:
train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
        '../data/cs231n_project/data/train',
        target_size=image_size,
        batch_size=batch_size,
        class_mode='categorical')
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = train_datagen.flow_from_directory(
        '../data/cs231n_project/data/val',
        target_size=image_size,
        batch_size=batch_size,
        class_mode='categorical')

Found 10177 images belonging to 9 classes.
Found 1328 images belonging to 9 classes.


In [4]:
nb_epoch = 30
nb_filters=32
kernel_size=(3,3)
pool_size=(2,2)

In [5]:
img=Input((3,224,224))
resnet_model = ResNet50(weights='imagenet',include_top=False)
print (resnet_model.output_shape)
middle=Input((2048,1,1))
x=Flatten()(middle)
x=Dense(128)(x)
x=Activation('relu')(x)
x=Dropout(0.5)(x)
x=Dense(nb_classes)(x)
x=Activation('softmax')(x)
top_model=Model(input=middle,output=x,name="topmodel")
model=Model(input=img,output=top_model(resnet_model(img)))

(None, 2048, None, None)


In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer='adadelta',
              metrics=['accuracy'])
#print(resent_model.summary())
#print(top_model.summary())
print(model.summary())

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 3, 224, 224)   0                                            
____________________________________________________________________________________________________
resnet50 (Model)                 multiple              23587712    input_1[0][0]                    
____________________________________________________________________________________________________
topmodel (Model)                 (None, 9)             263433      resnet50[1][0]                   
Total params: 23,851,145
Trainable params: 23,798,025
Non-trainable params: 53,120
____________________________________________________________________________________________________
None


In [ ]:
#first train all layers
trainall_hist=model.fit_generator(train_generator,samples_per_epoch=1000, 
                         validation_data=test_generator,nb_val_samples=100,
                         nb_epoch=nb_epoch,verbose=1)
trainall_score = model.evaluate_generator(test_generator, val_samples=100)
print('Test score:', trainall_score[0])
print('Test accuracy:', trainall_score[1])

Epoch 1/30
1000/1000 [==============================] - 91s - loss: 1.4656 - acc: 0.5610 - val_loss: 2.5368 - val_acc: 0.1058
Epoch 2/30
1000/1000 [==============================] - 90s - loss: 1.0290 - acc: 0.7100 - val_loss: 5.3723 - val_acc: 0.1058
Epoch 3/30
1000/1000 [==============================] - 90s - loss: 0.8137 - acc: 0.7560 - val_loss: 3.1098 - val_acc: 0.0769
Epoch 4/30
1000/1000 [==============================] - 91s - loss: 0.7542 - acc: 0.7730 - val_loss: 3.7843 - val_acc: 0.0769
Epoch 5/30
1000/1000 [==============================] - 90s - loss: 0.7068 - acc: 0.7750 - val_loss: 4.6630 - val_acc: 0.1538
Epoch 6/30
1000/1000 [==============================] - 90s - loss: 0.5795 - acc: 0.8260 - val_loss: 2.8741 - val_acc: 0.2981
Epoch 7/30
1000/1000 [==============================] - 89s - loss: 0.6869 - acc: 0.8070 - val_loss: 1.9105 - val_acc: 0.5288
Epoch 8/30
1000/1000 [==============================] - 89s - loss: 0.5735 - acc: 0.8460 - val_loss: 2.3272 - val_acc:

In [ ]:
# now try with resnet frozen
for layer in resnet_model.layers:
    layer.trainable=False

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer='adadelta',
              metrics=['accuracy'])
print(model.summary())

In [ ]:
#now just train the top layers
traintop_hist=model.fit_generator(train_generator,samples_per_epoch=1000, 
                         validation_data=test_generator,nb_val_samples=100,
                         nb_epoch=nb_epoch,verbose=1)
traintop_score = model.evaluate_generator(test_generator, val_samples=100)
print('Test score:', traintop_score[0])
print('Test accuracy:', traintop_score[1])

In [ ]:
#now try spit training, training top first, then resnet
#resnet frozen
for layer in resnet_model.layers:
    layer.trainable=False

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer='adadelta',
              metrics=['accuracy'])
print(model.summary())

In [ ]:
trainsplittop_hist=model.fit_generator(train_generator,samples_per_epoch=1000, 
                         validation_data=test_generator,nb_val_samples=100,
                         nb_epoch=nb_epoch/2,verbose=1)
trainsplittop_score = model.evaluate_generator(test_generator, val_samples=100)
print('Test score:', trainsplittop_score[0])
print('Test accuracy:', trainsplittop_score[1])

In [ ]:
top_model.save_weights("temp")

In [ ]:
#resnet frozen
for layer in resnet_model.layers:
    layer.trainable=True

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer='adadelta',
              metrics=['accuracy'])
top_model.load_weights("temp", by_name=True)
print(model.summary())

In [ ]:
model.load_weights("temp", by_name=True)

In [ ]:
trainsplitfull_hist=model.fit_generator(train_generator,samples_per_epoch=1000, 
                         validation_data=test_generator,nb_val_samples=100,
                         nb_epoch=nb_epoch/2,verbose=1)
trainsplitfull_score = model.evaluate_generator(test_generator, val_samples=10)
print('Test score:', trainsplitfull_score[0])
print('Test accuracy:', trainsplitfull_score[1])

In [ ]:

f, axarr = plt.subplots(3,figsize=(12, 12))
labelx=0
labely=.1
axarr[0].set_title('Train all layers')
axarr[0].text(labelx, labely, 'Test score: {}'.format(trainall_score[0]))
axarr[0].text(labelx, labely*2, 'Test accuracy: {}  '.format(trainall_score[1]))
axarr[0].text(labelx, labely*3, 'Max Train acc: {}'.format(np.max(trainall_hist.history['acc'])))
axarr[0].text(labelx, labely*4, 'Max Val   acc: {} '.format(np.max(trainall_hist.history['val_acc'])))
axarr[0].plot(trainall_hist.history['acc'])
axarr[0].plot(trainall_hist.history['val_acc'])

axarr[1].set_title('Train top layers')
axarr[1].text(labelx, labely, 'Test score: {}'.format(trainall_score[0]))
axarr[1].text(labelx, labely*2, 'Test accuracy: {}  '.format(trainall_score[1]))
axarr[1].text(labelx, labely*3, 'Max Train acc: {}'.format(np.max(traintop_hist.history['acc'])))
axarr[1].text(labelx, labely*4, 'Max Val   acc: {} '.format(np.max(traintop_hist.history['val_acc'])))
axarr[1].plot(traintop_hist.history['acc'])
axarr[1].plot(traintop_hist.history['val_acc'])

axarr[2].set_title('Train top,then all layers')
acc=np.concatenate([trainsplittop_hist.history['acc'],trainsplitfull_hist.history['acc']],axis=0)
val_acc=np.concatenate([trainsplittop_hist.history['val_acc'],trainsplitfull_hist.history['val_acc']],axis=0)
axarr[2].text(labelx, labely, 'Test score: {}'.format(trainsplitfull_score[0]))
axarr[2].text(labelx, labely*2, 'Test accuracy: {}  '.format(trainsplitfull_score[1]))
axarr[2].text(labelx, labely*3, 'Max Train acc: {}'.format(acc))
axarr[2].text(labelx, labely*4, 'Max Val   acc: {} '.format(val_acc))
axarr[2].plot(acc)
axarr[2].plot(val_acc)
